#### 前期准备

In [23]:
# 获取当前notebook的绝对路径
import sys, os
from pathlib import Path
current_path = Path.cwd()  # 或者使用 %pwd magic命令
project_root = current_path.parents[4]  # 如果notebook在project_root/notebooks/下
print("notebook所在的路径：", current_path)
print("project_root：", project_root)
sys.path.append(str(project_root))
import FastAPI_setup

notebook所在的路径： /home/oscarwang/BidPilot_new/bidlyzer-service/app/clients/tiptap/tools/tests
project_root： /home/oscarwang/BidPilot_new/bidlyzer-service


In [24]:
project_id = "20563b61-7818-474b-9641-c40757bbd3fe"  # test1
#project_id = "cc6bca8a-df5d-43d5-a931-875113df1f50"  # test2

In [25]:
from app.services.structuring.cache import Cache
cache = Cache(project_id)
raw_document = await cache.get_document('raw_document')
h1_document = await cache.get_document('h1_document')
h2h3_document = await cache.get_document('h2h3_document')
intro_document = await cache.get_document('intro_document')
final_document = await cache.get_document('final_document')

获取文档数据失败 final_document: cannot access local variable 'e' where it is not associated with a value


In [26]:
print("raw_document:",type(raw_document), len(raw_document))
print("h1_document:",type(h1_document), len(h1_document))
print("h2h3_document:",type(h2h3_document), len(h2h3_document))
print("intro_document:",type(intro_document), len(intro_document))
# print("final_document:",type(final_document), len(final_document))

raw_document: <class 'dict'> 2
h1_document: <class 'dict'> 2
h2h3_document: <class 'dict'> 2
intro_document: <class 'dict'> 2


#### 验证extracted_chapters_md_with_position

In [5]:
from app.clients.tiptap.tools import formatted_chapters_md_with_position
chapters_md = formatted_chapters_md_with_position(h1_document)
print(len(chapters_md))
from pprint import pprint
pprint(chapters_md[0])

6
{'content': '章节标题: 第一章 招标公告 | position: 75\n'
            'content: 根据《中华人民共和国招标投标法》、《中华人民共和国招标投标法实施条 例》等规定， '
            '北京中外建工程管理有限公司受招标人委托，就北京京铁运恒采购 供应站有限公司 2024 '
            '年端午节物资采购项目（第一包：一级压榨花生油）进行 国内公开招标采购，欢迎合格的投标人前来投标。 | position: 76\n'
            'content: 一、项目编号：DLXM-2024-148-01 | position: 77\n'
            'content: 二、采购组织类型：委托代理 | position: 78\n'
            'content: 三、采购方式：公开招标 | position: 79\n'
            'content: 四、招标最高限价：1840 万元 | position: 80\n'
            'content: 五、采购内容、数量及简要技术要求 | position: 81\n'
            'content:  | position: 82\n'
            '[table]: 表格内容此处省略... | position: 83\n'
            'content: 六、合格投标人的资格要求(本项目采用资格后审) | position: 84\n'
            'content: （1）在中华人民共和国境内注册的、具有独立承担民事责任的能力； | position: 85\n'
            'content: （2）具有依法缴纳税收和社会保险的良好记录； | position: 86\n'
            'content: （3）近三年内，在经营活动中没有重大违法记录； | position: 87\n'
            'content: （4）单位负责人为同一人或者存在直接控股、管理关系的不同投标人，不得 同时参加本项目的投标； | '
            'position: 88\n'
       

In [6]:
print(type(chapters_md[0]))
print(type(chapters_md[0]["content"]))

<class 'dict'>
<class 'str'>


#### 验证 extract_chapters_by_nodes


In [7]:
from app.clients.tiptap.tools import extract_chapters_by_nodes
chapters = extract_chapters_by_nodes(h2h3_document)
print(len(chapters))
chapter1 = chapters[0]
chapter2 = chapters[1]
chapter3 = chapters[2]
chapter4 = chapters[3]
chapter5 = chapters[4]
chapter6 = chapters[5]
print("chapter1:",type(chapter1), f"{len(chapter1['content'])} nodes")
print("chapter2:",type(chapter2), f"{len(chapter2['content'])} nodes")
print("chapter3:",type(chapter3), f"{len(chapter3['content'])} nodes")
print("chapter4:",type(chapter4), f"{len(chapter4['content'])} nodes")
print("chapter5:",type(chapter5), f"{len(chapter5['content'])} nodes")
print("chapter6:",type(chapter6), f"{len(chapter6['content'])} nodes")


6
chapter1: <class 'dict'> 54 nodes
chapter2: <class 'dict'> 37 nodes
chapter3: <class 'dict'> 208 nodes
chapter4: <class 'dict'> 78 nodes
chapter5: <class 'dict'> 191 nodes
chapter6: <class 'dict'> 284 nodes


In [22]:
from app.clients.tiptap.client import TiptapClient
chapter1_html = await TiptapClient().json_to_html(chapter1)
from IPython.display import display, HTML
display(HTML(chapter1_html))


招标内容,规格,数量 （桶）,最高预算单价(含税价),交货时间
一级压榨花 生油,5L/桶,160000,115 元/桶,中标人接到中标通知书开始备 货，签订合同后开始送货，实际 送货进度以招标人送货计划通 知为准。


#### 验证add_introduction_headings， 使用chapter2的案例，该章节没有前言内容


In [9]:
from app.clients.tiptap.tools import get_paragraph_nodes_with_position
paragraphs = get_paragraph_nodes_with_position(chapter2)
print(len(paragraphs))
from pprint import pprint
pprint(paragraphs)


34
[{'node': {'attrs': {'level': 1, 'textAlign': 'left'},
           'content': [{'text': '第二章 招标需求', 'type': 'text'}],
           'type': 'heading'},
  'position': 0,
  'type': 'heading'},
 {'node': {'attrs': {'level': 2, 'textAlign': 'left'},
           'content': [{'text': '一、概述', 'type': 'text'}],
           'type': 'heading'},
  'position': 1,
  'type': 'heading'},
 {'node': {'attrs': {'textAlign': 'left'},
           'content': [{'text': '1.本招标项目是针对北京京铁运恒采购供应站有限公司所需货物的采购、供应、 '
                                '运输、验收及售后服务进行的招标。',
                        'type': 'text'}],
           'type': 'paragraph'},
  'position': 2,
  'type': 'paragraph'},
 {'node': {'attrs': {'textAlign': 'left'},
           'content': [{'text': '2.本项目的组织及解释权归招标人。', 'type': 'text'}],
           'type': 'paragraph'},
  'position': 3,
  'type': 'paragraph'},
 {'node': {'attrs': {'textAlign': 'left'},
           'content': [{'text': '3.招标文件中的技术要求是基本质量要求，投标人应根据自己的投标方案综合考 '
                                '虑详细的技术指标

In [10]:
from app.clients.tiptap.tools import get_paragraph_text_with_position
paragraphs = get_paragraph_text_with_position(chapter2)
print(len(paragraphs))
from pprint import pprint
pprint(paragraphs)

34
[{'content': '第二章 招标需求', 'position': 0, 'type': 'heading'},
 {'content': '一、概述', 'position': 1, 'type': 'heading'},
 {'content': '1.本招标项目是针对北京京铁运恒采购供应站有限公司所需货物的采购、供应、 运输、验收及售后服务进行的招标。',
  'position': 2,
  'type': 'paragraph'},
 {'content': '2.本项目的组织及解释权归招标人。', 'position': 3, 'type': 'paragraph'},
 {'content': '3.招标文件中的技术要求是基本质量要求，投标人应根据自己的投标方案综合考 虑详细的技术指标，但不得低于招标基本质量要求。',
  'position': 4,
  'type': 'paragraph'},
 {'content': '4.投标人供应的产品，需符合国家《中华人民共和国产品质量法》、《中华人民 '
             '共和国食品安全法》等相关法律规定的要求，符合卫生防疫部门、检验检疫部门 '
             '的各项合格指标，并有完整的销售合格标注或证书,所有的产品可追溯供货源头， '
             '在行业标准、国家标准等各项标准不一致时应以要求高的标准为准。',
  'position': 5,
  'type': 'paragraph'},
 {'content': '5.投标人提供的所有货物都应具有合格证及质量检测报告。', 'position': 6, 'type': 'paragraph'},
 {'content': '', 'position': 7, 'type': 'paragraph'},
 {'content': '二、采购内容', 'position': 8, 'type': 'heading'},
 {'content': '', 'position': 9, 'type': 'paragraph'},
 {'content': '三、技术和质量要求', 'position': 11, 'type': 'heading'},
 {'content': '花生油质量不低于《花生油

In [11]:
from app.clients.tiptap.tools import add_introduction_headings
chapter2_intro = add_introduction_headings(chapter2)
print(len(chapter2_intro))
from pprint import pprint
pprint(chapter2_intro)

2
{'content': [{'attrs': {'level': 1, 'textAlign': 'left'},
              'content': [{'text': '第二章 招标需求', 'type': 'text'}],
              'type': 'heading'},
             {'attrs': {'level': 2, 'textAlign': 'left'},
              'content': [{'text': '一、概述', 'type': 'text'}],
              'type': 'heading'},
             {'attrs': {'textAlign': 'left'},
              'content': [{'text': '1.本招标项目是针对北京京铁运恒采购供应站有限公司所需货物的采购、供应、 '
                                   '运输、验收及售后服务进行的招标。',
                           'type': 'text'}],
              'type': 'paragraph'},
             {'attrs': {'textAlign': 'left'},
              'content': [{'text': '2.本项目的组织及解释权归招标人。', 'type': 'text'}],
              'type': 'paragraph'},
             {'attrs': {'textAlign': 'left'},
              'content': [{'text': '3.招标文件中的技术要求是基本质量要求，投标人应根据自己的投标方案综合考 '
                                   '虑详细的技术指标，但不得低于招标基本质量要求。',
                           'type': 'text'}],
              'type': 'paragraph'},
           

#### 验证add_introduction_headings， 使用chapter3的案例，该章节有前言内容

In [12]:
# 获取chapter3的段落节点
from app.clients.tiptap.tools import get_paragraph_nodes_with_position
paragraphs = get_paragraph_nodes_with_position(chapter3)
print(len(paragraphs))
from pprint import pprint
pprint(paragraphs)

203
[{'node': {'attrs': {'level': 1, 'textAlign': 'left'},
           'content': [{'text': '第三章 投标人须知', 'type': 'text'}],
           'type': 'heading'},
  'position': 0,
  'type': 'heading'},
 {'node': {'attrs': {'textAlign': 'left'},
           'content': [{'marks': [{'type': 'bold'}],
                        'text': '前附表',
                        'type': 'text'}],
           'type': 'paragraph'},
  'position': 1,
  'type': 'paragraph'},
 {'node': {'attrs': {'textAlign': 'left'}, 'type': 'paragraph'},
  'position': 2,
  'type': 'paragraph'},
 {'node': {'attrs': {'textAlign': 'left'}, 'type': 'paragraph'},
  'position': 4,
  'type': 'paragraph'},
 {'node': {'attrs': {'textAlign': 'left'}, 'type': 'paragraph'},
  'position': 5,
  'type': 'paragraph'},
 {'node': {'attrs': {'textAlign': 'left'}, 'type': 'paragraph'},
  'position': 6,
  'type': 'paragraph'},
 {'node': {'attrs': {'textAlign': 'left'}, 'type': 'paragraph'},
  'position': 8,
  'type': 'paragraph'},
 {'node': {'attrs': {'textA

In [13]:
# 添加前言标题
from app.clients.tiptap.tools import add_introduction_headings
chapter3_intro = add_introduction_headings(chapter3)
print(len(chapter3_intro))
from pprint import pprint
pprint(chapter3_intro)

2
{'content': [{'attrs': {'level': 1, 'textAlign': 'left'},
              'content': [{'text': '第三章 投标人须知', 'type': 'text'}],
              'type': 'heading'},
             {'attrs': {'level': 2},
              'content': [{'text': '前言', 'type': 'text'}],
              'type': 'heading'},
             {'attrs': {'textAlign': 'left'},
              'content': [{'marks': [{'type': 'bold'}],
                           'text': '前附表',
                           'type': 'text'}],
              'type': 'paragraph'},
             {'attrs': {'textAlign': 'left'}, 'type': 'paragraph'},
             {'content': [{'content': [{'attrs': {'colspan': 1,
                                                  'colwidth': None,
                                                  'rowspan': 1},
                                        'content': [{'attrs': {'textAlign': 'left'},
                                                     'content': [{'marks': [{'type': 'bold'}],
                                        

In [14]:
# 将chapter3_intro转换为html， 并看结果
from app.clients.tiptap.client import TiptapClient
intro_doc_html = await TiptapClient().json_to_html(chapter3_intro)
from IPython.display import display, HTML
display(HTML(intro_doc_html))


#### 验证add_introduction_headings， 使用h2h3_document的完整案例

In [15]:
# 添加前言标题
from app.clients.tiptap.tools import add_introduction_headings
h2h3_intro = add_introduction_headings(h2h3_document)
print(len(h2h3_intro))
from pprint import pprint
pprint(h2h3_intro)

2
{'content': [{'attrs': {'textAlign': 'left'}, 'type': 'paragraph'},
             {'attrs': {'textAlign': 'left'}, 'type': 'paragraph'},
             {'attrs': {'textAlign': 'left'}, 'type': 'paragraph'},
             {'attrs': {'textAlign': 'left'}, 'type': 'paragraph'},
             {'attrs': {'textAlign': 'left'}, 'type': 'paragraph'},
             {'attrs': {'textAlign': 'left'},
              'content': [{'marks': [{'type': 'bold'}],
                           'text': '北京京铁运恒采购供应站有限公司',
                           'type': 'text'},
                          {'text': ' ', 'type': 'text'},
                          {'marks': [{'type': 'bold'}],
                           'text': '2024',
                           'type': 'text'},
                          {'text': ' ', 'type': 'text'},
                          {'marks': [{'type': 'bold'}],
                           'text': '年端午节物资采购项目',
                           'type': 'text'}],
              'type': 'paragraph'},
             {'

In [16]:
# 将h2h3_intro转换为html， 并看结果
from app.clients.tiptap.client import TiptapClient
intro_doc_html = await TiptapClient().json_to_html(h2h3_intro)
from IPython.display import display, HTML
display(HTML(intro_doc_html))

In [17]:
# 打印标题列表看结果
from app.clients.tiptap.tools import get_headings
_,print_headings = get_headings(h2h3_intro)
print(print_headings)

[H1] 第一章 招标公告
  [H2] 前言
  [H2] 项目编号
  [H2] 采购组织类型
  [H2] 采购方式
  [H2] 招标最高限价
  [H2] 采购内容、数量及简要技术要求
  [H2] 合格投标人的资格要求(本项目采用资格后审)
  [H2] 招标公告的发布
  [H2] 招标文件的发售
  [H2] 投标保证金
  [H2] 投标截止时间和地点
  [H2] 开标时间及地点
  [H2] 评标办法
  [H2] 业务咨询
[H1] 第二章 招标需求
  [H2] 一、概述
  [H2] 二、采购内容
  [H2] 三、技术和质量要求
    [H3] 前言
    [H3] 2.花生油主要技术参数
    [H3] 4.样品
    [H3] *5. 送达地点
    [H3] *6. 交货时间
  [H2] 四、履约保证金
[H1] 第三章 投标人须知
  [H2] 前言
  [H2] 一、总则
    [H3] （一）适用范围
    [H3] （二） 定义
    [H3] （三） 招标方式
    [H3] （四）投标委托
    [H3] （五） 踏勘现场（本项目不适用）和投标费用
    [H3] （六） 联合体投标
    [H3] （七）转包
    [H3] （八）、特别说明：
    [H3] （九） 质疑和投诉
  [H2] 二、招标文件
    [H3] （一）招标文件的构成。本招标文件由以下部分组成：
    [H3] （二） 投标人的风险
    [H3] （三） 招标文件的澄清与修改
    [H3] （四）、对招标文件的异议
  [H2] 三、投标文件的编制
    [H3] （一）、投标文件的组成
    [H3] （二）、投标文件的语言及计量
    [H3] （三）、投标报价
    [H3] （四）、投标文件的有效期
    [H3] （五）、投标保证金
    [H3] （六）、投标文件的签署和份数
    [H3] （七）、投标文件的包装、递交、修改和撤回
    [H3] （八）、投标无效的情形
  [H2] 四、开标
    [H3] （一）、开标准备
    [H3] （二）、开标程序：
  [H2] 五、评标
    [H3] （一）、组建评标委员会
    [H3] （二）、评标的方式


### 验证 extract_introductions_by_nodes

In [18]:
print(f"final_document: {final_document}")
pprint(final_document)

final_document: None
None


In [19]:
from app.clients.tiptap.tools import extract_introductions_by_nodes
introductions = extract_introductions_by_nodes(h2h3_document)
print(len(introductions))
from pprint import pprint
pprint(introductions)

0
[]


In [20]:
import json
# 创建文档的深拷贝，避免修改原始文档
updated_doc = json.loads(json.dumps(chapter2))

# 查找所有标题及其位置
headings, _ = get_headings(updated_doc)

# 按文档顺序排序（根据位置）
headings.sort(key=lambda h: h["position"])

print(len(headings))
pprint(headings)

9
[{'level': 1,
  'node': {'attrs': {'level': 1, 'textAlign': 'left'},
           'content': [{'text': '第二章 招标需求', 'type': 'text'}],
           'type': 'heading'},
  'position': 0,
  'title': '第二章 招标需求'},
 {'level': 2,
  'node': {'attrs': {'level': 2, 'textAlign': 'left'},
           'content': [{'text': '一、概述', 'type': 'text'}],
           'type': 'heading'},
  'position': 1,
  'title': '一、概述'},
 {'level': 2,
  'node': {'attrs': {'level': 2, 'textAlign': 'left'},
           'content': [{'text': '二、采购内容', 'type': 'text'}],
           'type': 'heading'},
  'position': 8,
  'title': '二、采购内容'},
 {'level': 2,
  'node': {'attrs': {'level': 2, 'textAlign': 'left'},
           'content': [{'text': '三、技术和质量要求', 'type': 'text'}],
           'type': 'heading'},
  'position': 11,
  'title': '三、技术和质量要求'},
 {'level': 3,
  'node': {'attrs': {'level': 3, 'textAlign': 'left'},
           'content': [{'text': '2.花生油主要技术参数', 'type': 'text'}],
           'type': 'heading'},
  'position': 13,
  'title': '

In [21]:
from app.clients.tiptap.tools import extract_text_from_node, get_headings